In [1]:
import pandas as pd
import numpy as np
import pickle
import heapq

In [2]:
###----------------------FILEPATHS----------------------###
primary_facility_path = '../Data/Primary_Facility_50.csv'
secondary_facility_path = '../Data/Secondary_Facility_500.csv'

supplier_path = '../Data/Supplier.csv'
# result_path = '../Results/Agglomerative_clustering_rq_200.pkl'

result_path = '../Results/Agglomerative_clustering_rq_50.pkl'

customer_path = '../Results/50_customer_coordinates.npy'
# customer_path = '../Results/200_customer_coordinates.npy'


###----------------------PARAMETERS----------------------###
RP = 5
seed = 100
counter = 0
C_SUPPLIER = 0.0006
C_PRIMARY = 0.0024
C_CUSTOMER = 0.012
MAX_ITERATIONS = 100

In [3]:
###----------------------READ ALL INPUT DATAFILES----------------------###
p_df = pd.read_csv(primary_facility_path)
q_df = pd.read_csv(secondary_facility_path)
s_df = pd.read_csv(supplier_path)
with open(result_path, 'rb') as f:
    vq_k_dict = pickle.load(f)

k_coordinates = np.load(customer_path)

In [4]:
###----------------------DIJKSTRA'S ALGORITHM FOR E-STEP : TO FIND SHORTEST PATHS BETWEEN SUPPLIER TO PRIMARY FACILITY TO SECONDARY FACILITIES----------------------###
def expectation_algorithm(graph, starts):
    shortest_path = {node: float('infinity') for node in graph}
    prev = {node: None for node in graph}
    priority_queue = []

    ###----------------------INITIALIZE ALL POINTS WITH A DISTANCE = 0----------------------###
    for start in starts:
        shortest_path[start] = 0
        heapq.heappush(priority_queue, (0, start))

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > shortest_path[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            if distance < shortest_path[neighbor]:
                shortest_path[neighbor] = distance
                prev[neighbor] = current_node  # FOR PATH RE-CONSTRUCTION
                heapq.heappush(priority_queue, (distance, neighbor))

    return shortest_path, prev

def reconstruct_path(source, target, prev):
    path = [target]
    while target in prev and prev[target] is not None:
        target = prev[target]
        path.append(target)
    path.reverse()
    return path


In [5]:
secondary_to_customers_idx = []
secondary_facilities_idx = []

for Q, K in vq_k_dict.items():
    q_idx = int(Q.split('_')[1])
    k_idx = int(K.split('_')[1])
    secondary_to_customers_idx.append([q_idx, k_idx])   # LOAD SECONDARY FACILITIES TO CUSTOMER COMPUTED FROM AGGLOMERATIVE CLUSTERING
    secondary_facilities_idx.append(q_idx)  # LOAD SECONDARY FACILITIY INDICES

p_df_sampled = p_df.sample(RP, random_state=seed)   # RANDOMLY SAMPLING INITIAL PRIMARY FACILITIES WITH A DEFINED SEED
primary_facilities_idx = list(p_df_sampled.index)   # LOAD INITIAL PRIMARY FACILITY INDICES
supplier_facilties_idx = list(s_df.index)   # LOAD SUPPLIER INDICES

all_primary_facilities_dict = dict(zip([p for p in list(p_df.index)], p_df[['Latitude', 'Longitude']].values))  # PRIMARY FACILTY SET {INDEX : COORDINATES}

updated_primary_list = []   # INITIALIZE THE PRIMARY FACILITY INDEX LIST TO BE UPDATED IN E-M STEP

In [6]:
###----------------------ITERATIVELY SOLVING E-M ALGORITHM----------------------###
print(f'INITIAL RANDOM PRIMARY FACILITIES : ', list(p_df_sampled.index))
while counter < MAX_ITERATIONS:

    ###----------------------DEFINE GRAPH----------------------###
    graph = {}

    ###----------------------SUPPLIER TO PRIMARY----------------------###
    for s in s_df['Index']:
        graph[f'S_{s}'] = {}
        for p in primary_facilities_idx:
            src = s_df.loc[s][['Latitude', 'Longitude']].values
            dest = all_primary_facilities_dict[p]
            graph[f'S_{s}'][f'P_{p}'] = np.linalg.norm(src - dest) * C_SUPPLIER

    ###----------------------PRIMARY TO SECONDARY----------------------###
    for p in primary_facilities_idx:
        graph[f'P_{p}'] = {}
        for q in secondary_facilities_idx:
            src = all_primary_facilities_dict[p]
            dest = q_df.loc[q][['Latitude', 'Longitude']].values
            graph[f'P_{p}'][f'Q_{q}'] = np.linalg.norm(src - dest) * C_PRIMARY

    ## REMARK : We assume Secondary facilities as the destination nodes that are must to be fulfilled from Supplier
    for q in secondary_facilities_idx:
        graph[f'Q_{q}'] = {}

    src = f'S_{supplier_facilties_idx[0]}'

    ###----------------------E-STEP----------------------###
    shortest_distances, previous_vertices = expectation_algorithm(graph, [f'S_{s}' for s in s_df['Index']])
    all_shortest_paths = {q: reconstruct_path(src, q, previous_vertices) for q in [f'Q_{i}' for i in secondary_facilities_idx]}

    ###----------------------INITIALIZATION OF X, Y, T FLOWS----------------------###
    x_flows = {}  # x_{si} flows
    y_flows = {}  # y_{ij} flows
    tau_flows = {}  # t_{ii'} flows}

    for sec_facility, path in all_shortest_paths.items():
        ###----------------------EXTRACT SUPPLIER, PRIMARY AND SECONDARY PATHS FROM SHORTEST PATHS COMPUTED IN E-STEP----------------------###
        supplier, primary, secondary = path
        ###----------------------COMPUTE Xsp_FLOW----------------------###
        x_flows_key = (supplier, primary)
        if x_flows_key in x_flows:
            x_flows[x_flows_key] += 1
        else:
            x_flows[x_flows_key] = 1

        ###----------------------COMPUTE Ypq_FLOW----------------------###
        y_flows_key = (primary, secondary)
        if y_flows_key in y_flows:
            y_flows[y_flows_key] += 1
        else:
            y_flows[y_flows_key] = 1

    ###----------------------DETERMINE T_FLOW USING NEAREST NEIGHBORS TO PRIMARY FACILITIES----------------------###
    nearest_neighbors = {}
    for p1 in all_primary_facilities_dict.keys():
        closest_p = None
        closest_distance = float('inf')
        for p2 in all_primary_facilities_dict.keys():
            if p1 != p2:
                distance = np.linalg.norm(all_primary_facilities_dict[p1] - all_primary_facilities_dict[p2])
                if distance < closest_distance:
                    closest_distance = distance
                    closest_p = p2
        nearest_neighbors[p1] = closest_p

    ###----------------------COMPUTE Tpp'_FLOW----------------------###

    for primary, neighbor in nearest_neighbors.items():
        for secondary, flow in y_flows.items():
            if secondary[0] == primary:
                key = (primary, neighbor)
                if key in tau_flows:
                    tau_flows[key] += flow
                else:
                    tau_flows[key] = flow
    
    ###--------------------------------------------------###
    ###----------------------M-STEP----------------------###
    ###--------------------------------------------------###

    ###----------------------INITIALIZE T, A, B MATRIX FOR OPTIMAL COORDINATES COMPUTATION----------------------###
    T = np.zeros((RP, RP))
    A = np.zeros(RP)
    B = np.zeros(RP)
    # print(T)

    ###----------------------COMPUTE T-MATRIX----------------------###
    for p in range(RP):
        for p_ in range(RP):
            if p == p_:
                ###----------------------COMPUTE DIAGONAL ELEMENTS IN T-MATRIX----------------------###
                ###----------------------REMARK : ADDED THE 1e-10 TERM TO ENSURE THAT T_inv ALWAYS EXIST 
                T[p][p_] = 1e-10 + (C_SUPPLIER * sum(x_flows.get((f'S_{s}', f'P_{primary_facilities_idx[p]}'), 0) for s in supplier_facilties_idx)
                        + C_PRIMARY * sum(tau_flows.get((f'P_{primary_facilities_idx[p]}', f'P_{primary_facilities_idx[p_prime]}'), 0) + tau_flows.get((f'P_{primary_facilities_idx[p_prime]}', f'P_{primary_facilities_idx[p]}'), 0) for p_prime in range(RP) if p_prime != p)
                        + C_PRIMARY * sum(y_flows.get((f'P_{primary_facilities_idx[p]}', f'Q_{q}'), 0) for q in secondary_facilities_idx))
                # print(T)
            else:
                ###----------------------COMPUTE OFF-DIAGONAL T-MATRIX----------------------###
                T[p][p_] = -C_PRIMARY * (tau_flows.get((f'P_{primary_facilities_idx[p]}', f'P_{primary_facilities_idx[p_]}'), 0) + tau_flows.get((f'P_{primary_facilities_idx[p_]}', f'P_{primary_facilities_idx[p]}'), 0))
                # print(T)
    ###----------------------COMPUTE A AND B VECTORS----------------------###
    for p in range(RP):
        A[p] = C_SUPPLIER * sum(x_flows.get((s, f'P_{primary_facilities_idx[p]}'), 0) * s_df.loc[s]['Latitude'] for s in supplier_facilties_idx) + C_PRIMARY * sum(y_flows.get((f'P_{primary_facilities_idx[p]}', f'Q_{q}'), 0) * q_df.loc[q]['Latitude'] for q in secondary_facilities_idx)
        B[p] = C_SUPPLIER * sum(x_flows.get((s, f'P_{primary_facilities_idx[p]}'), 0) * s_df.loc[s]['Longitude'] for s in supplier_facilties_idx) + C_PRIMARY * sum(y_flows.get((f'P_{primary_facilities_idx[p]}', f'Q_{q}'), 0) * q_df.loc[q]['Longitude'] for q in secondary_facilities_idx)

    ###----------------------COMPUTE OPTIMAL PRIMARY FACILITY COORDINATES----------------------###
    T_inv = np.linalg.inv(T)
    calculated_coords = {}
    for p in range(RP):
        calculated_coords[f'P_{primary_facilities_idx[p]}'] = (np.dot(T_inv[p], A), np.dot(T_inv[p], B))
    
    ###----------------------COMPUTE NEAREST PRIMARY FACILITY FROM THE OPTIMAL COORDINATES----------------------###
    for coords in calculated_coords.values():
        nearest_dist = float('infinity')
        for p_idx, p_coords in all_primary_facilities_dict.items():
            dist = np.linalg.norm(coords - p_coords)
            if dist < nearest_dist and p_idx not in set(updated_primary_list):
                nearest_dist = dist
                nearest_coords = p_coords
                nearest_idx = p_idx
        updated_primary_list.append(nearest_idx)

    ###----------------------TERMINATION CRITERION----------------------###
    if set(updated_primary_list) == set(primary_facilities_idx):
        print(f'FINAL BEST PRIMARY FACILITIES : {updated_primary_list}')
        for s, path in all_shortest_paths.items():
            print(f"Shortest path to {s}: {path} and cost : {shortest_distances[s]}")
        break
    
    ###----------------------ELSE CONTINUE TILL WHILE LOOP TERMINATES----------------------###
    else:
        primary_facilities_idx = list(updated_primary_list)
        counter += 1
        print(f'COUNTER : {counter}')
        print(f'CURRENT BEST PRIMARY FACILITIES : {primary_facilities_idx}')
        updated_primary_list = []

INITIAL RANDOM PRIMARY FACILITIES :  [6, 36, 37, 28, 43]
COUNTER : 1
CURRENT BEST PRIMARY FACILITIES : [17, 20, 28, 14, 8]
COUNTER : 2
CURRENT BEST PRIMARY FACILITIES : [20, 14, 17, 41, 23]
FINAL BEST PRIMARY FACILITIES : [20, 14, 17, 41, 23]
Shortest path to Q_65: ['S_0', 'P_23', 'Q_65'] and cost : 0.03491389764381461
Shortest path to Q_440: ['S_0', 'P_23', 'Q_440'] and cost : 0.10118775038956254
Shortest path to Q_338: ['S_0', 'P_17', 'Q_338'] and cost : 0.03715485067035948
Shortest path to Q_51: ['S_0', 'P_23', 'Q_51'] and cost : 0.030414069229470396
Shortest path to Q_161: ['S_0', 'P_17', 'Q_161'] and cost : 0.025044425244849643
Shortest path to Q_317: ['S_0', 'P_17', 'Q_317'] and cost : 0.021599521921081256
Shortest path to Q_50: ['S_0', 'P_20', 'Q_50'] and cost : 0.027558324626371457
Shortest path to Q_237: ['S_0', 'P_41', 'Q_237'] and cost : 0.055431845813842126
Shortest path to Q_31: ['S_0', 'P_23', 'Q_31'] and cost : 0.10541439852959403
Shortest path to Q_332: ['S_0', 'P_17', 